## nonplanar spacer

In [ ]:
import fullcontrol as fc
from math import cos, tau

In [ ]:
# design parameters

# main design parameters
D1 = 8 # inner diameter
total_thickness = 4  # intended thickness of washer # try 3 or 4
waves = 6  # number of fluctuating waves in z direction

# advanced parameters
D_ratio = 3 # ratio between inner and outer diameters
material_thickness = 0.4
EW_EH_ratio = 2
overlap_percent = 20 # overlap between filaments
contraction_factor = 1.2 # this means the XY line separation reduces when the nozzle also moves up in z. try 1 or 1.2
quantity = 1

In [ ]:
# create design steps

EH = material_thickness
EW = EH*EW_EH_ratio
D2 = D1*D_ratio 
overlap = (overlap_percent/100) * EW 
# increase in Z height of nozzle to get desired total_thickness:
height = total_thickness - EH 
R1, R2 = D1/2, D2/2
rings = int((R2-R1)/(EW-overlap))
segs_per_ring = (waves*2)*int(128/(waves*2)) # this means the segmented path always has a node at the exact min and max of waves

centre = fc.Point(x=0, y=0, z=0)
# set start point, and travel-line into centre of part:
steps = [fc.move_polar(centre, centre, R2, (0.5+((waves+1)%2)/(2*waves))*tau), fc.Extruder(on=False), centre, fc.Extruder(on=True)]
# add spiral purge line (if there is space):
purge_spiral_passes = min(int((R1-EW)/EW)-1, 3)
if purge_spiral_passes > 0: steps.extend(fc.spiralXY(centre, EW/2, R1-EW, 0, purge_spiral_passes, 200))

# print part:
for ring in range(rings):
  for seg in range(segs_per_ring+1):  # need one extra 'seg' to allow for the first segment having a start point as well as an end point
    angle_now = (seg/segs_per_ring)*tau
    z_now = height*(ring/(rings-1))*(0.5-0.5*cos(angle_now*waves))
    radius_now = R1 + EW/2 + ring*(EW-overlap)-(z_now*contraction_factor)
    centre.z = z_now
    steps.append(fc.polar_to_point(centre, radius_now, angle_now))
    
# print multiple copies:
if quantity > 1: steps = fc.move(steps, fc.Vector(x=R2*2 + 5), copy=True, copy_quantity=quantity)

# offset the whole procedure. z dictates the gap between the nozzle and the bed for the first layer, assuming the model was designed with a first layer z-position of 0
model_offset = fc.Vector(x=50, y=50, z=0.8*EH)
steps = fc.move(steps, model_offset)

In [ ]:
# add annotations and plot

annotations = []
annotations.append(fc.PlotAnnotation(point=fc.midpoint(steps[0],steps[2]), label = "Initial approach set under a wave-crest to avoid defects"))
annotations.append(fc.PlotAnnotation(point=steps[0], label = "Start"))
annotations.append(fc.PlotAnnotation(point=steps[-1], label = "End"))
annotations.append(fc.PlotAnnotation(point=fc.move(steps[2], fc.Vector(z=total_thickness*2)), label="A pointy nozzle is best"))
if purge_spiral_passes>0: 
    annotations.append(fc.PlotAnnotation(point=steps[2], label="Spiral flow stabiliser"))
if quantity > 1:
    annotations.append(fc.PlotAnnotation(point=fc.move(centre, fc.Vector(x=model_offset.x+(R2*2 + 5),
                       y=model_offset.y, z=model_offset.z)), label="Designed movement between parts and spiral-purge each time"))
    
fc.transform(steps + annotations, 'plot', fc.PlotControls(color_type='print_sequence'))

In [ ]:
# generate and save gcode
initial_settings = {
    "print_speed": 500,
    "nozzle_temp": 210,
    "bed_temp": 40,
    "fan_percent": 100,
    "extrusion_width": EW,
    "extrusion_height": EH
    }
gcode_controls = fc.GcodeControls(printer_name='bambulab_x1', initialization_data=initial_settings, save_as = "nonplanar_spacer")
fc.transform(steps, 'gcode', gcode_controls)